# Measuring Boyer-Moore's benefit

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [5]:
# Import boyer moore preprocessing module
import bm_preproc

In [6]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2022-03-03 13:54:57--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.220.151, 13.226.220.77, 13.226.220.74, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.220.151|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  2.24MB/s    in 0.3s    

2022-03-03 13:54:58 (2.24 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [7]:
# parses a DNA reference genome from a file in the FASTA format.
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [29]:
# Naive exact matching that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried.
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        num_alignments += 1
        match = True
        for j in range(len(p)):  # loop over characters
            num_character_comparisons += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, num_alignments, num_character_comparisons

In [30]:
# Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried.
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

# Approximate Matching with Boyer Moore


In [28]:
#
# Practical: partial matching algroithm implementation using an exact matching algorithm (Boyer-Moore)
# combined with the pigeon hole principle to allow up to n mismatches of pattern in text
#
def approximate_match_boyer_moore(p, t, n):
    segment_length = int(round(len(p) // (n+1)))
    all_matches = set()
    for i in range(n+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches, _, _ = boyer_moore(p[start:end], p_bm, t)
        for m in matches:
            #
            # Question: why is text offset equal to the matched position minus start of current segment?
            #
            # Answer: this is beacuse 'start' is the n-th segment offset within p, so we are basically aligning pattern within text based on the n-th exact matching segment
            # so text offset from T's perspective is where P would begin to align within T.
            #
            # Example:
            #
            #       01234567
            #   T = ABCDEFG
            #   P =   CDEF
            #   N = 1
            #
            # since N = 1, then N + 1 = 2, thus P is split into 2 partitions CD and EF
            # the first partition:  CD matches T at m = 2, the start of partition CD is 0, so P aligns with T at m = 2 - 0 = 2
            # the second partition: EF matches T at m = 4, the start of partition EF is 2, so P aligns with T at m = 4 - 2 = 2
            # thus for each match in a k-th partition, P aligns at same offset m within T ( ie. offset 2 for this example )
            #
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(text_offset)
    return list(all_matches)

# Approximate Matching with Indexing

In [41]:
# a Python class called Index implementing an ordered-list version of the k-mer index.
import bisect
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [49]:
#
# partial matching algroithm implementation using an exact matching algorithm (Indexing)
# combined with the pigeon hole principle to allow up to n mismatches of pattern in text
#
def approximate_match_index(p, t, n):
    segment_length = int(round(len(p) // (n+1)))
    hits = 0
    all_matches = set()
    index = Index(t, 8) # built on 8-mers
    for i in range(n+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))
        matches = index.query(p[start:end])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(text_offset)
    return list(all_matches), hits

# Approximate Matching with Subsequence Indexing

In [60]:
# The following class \verb|SubseqIndex|SubseqIndex is a more general implementation of Index that additionally handles subsequences. 
# It only considers subsequences that take every Nth character:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [61]:
# example 
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [62]:
# p = 'TTATAT' # we will get [] when we run because the subsequence TAA is not in the index. But if we query with the second subsequence:
print(ind.query(p[0:]))


[]


In [64]:
print(ind.query(p[1:])) # because the second subsequence TTT is in the index.

[]


In [48]:
#
# partial matching algroithm implementation using an exact matching algorithm (Subsequence Indexing)
# combined with the pigeon hole principle to allow up to k mismatches of pattern in text
#
# note: in this solution, the partitions overlap ( ie. every 3rd subsequence interval, 123123123 )
# compared to the previous solutions where each partition has a unique, non-overlapping start/end
# ( ie. every 3rd chunk, 111222333 )
#
def approximate_match_subseq_index(p, t, n):
    segment_length = int(round(len(p) // (n+1)))
    hits = 0
    all_matches = set()
    index = SubseqIndex(t, 8, 3) # built on 8-mers and subsequence intervals of 3
    for start in range(n+1):
        matches = index.query(p[start:])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(text_offset)
    return list(all_matches), hits

In [14]:
chr1_genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [31]:
# Example 1

p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [32]:
# Example 2
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [33]:
from bm_preproc import BoyerMoore
# Example 1
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [34]:
# Example 2
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


In [35]:
# Question 1
# How many alignments does the naive exact matching algorithm try when matching the string 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG' 
# (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

# Question 2
# How many character comparisons does the naive exact matching algorithm try when matching the string 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG' 
# (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = chr1_genome
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [36]:
# Question 3 
# How many alignments does Boyer-Moore try when matching the string 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG' 
# (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = chr1_genome
p_bm = BoyerMoore(p, alphabet='ACGT')
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


In [51]:
# Question 4
# How many times does the string 'GGCGCGGTGGCTCACGCCTGTAAT', 
# which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

# Question 5
# Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 'GGCGCGGTGGCTCACGCCTGTAAT' with up to 2 substitutions in the excerpt of human chromosome 1?
# (Don't consider reverse complements.)


p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = chr1_genome
n = 2
all_matches, hits = approximate_match_index(p, t, n)
print(all_matches, hits)

[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299] 90


In [52]:
# naive exact matching algorithm
# Returns a list of occurrences (offset)
def naive_2mm(p, t):
    occurrences = []

    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch_count = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatch_count+=1

                if mismatch_count > 2:
                	match = False
                	break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [57]:
# cross check question 4 with naive_2mm function 
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = chr1_genome
occurrences = naive_2mm(p,t)
print(occurrences)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [58]:
# Question 5
# Hint: You should be able to use the 'boyer_moore' function (or the slower 'naive' function) to double-check your answer.

p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = chr1_genome
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922, 262042, 364263, 657496, 717706] 799977 984116


In [59]:
# Question 5
# Hint: You should be able to use the 'boyer_moore' function (or the slower 'naive' function) to double-check your answer.

p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = chr1_genome
p_bm = BoyerMoore(p, alphabet='ACGT')
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922, 262042, 364263, 657496, 717706] 126203 196873


In [65]:
# Question 6
# Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.
# When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = chr1_genome
n = 2
all_matches, hits = approximate_match_subseq_index(p, t, n)
print(all_matches, hits)

[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299] 79
